In [4]:
from skimage import io
from skimage.color import rgb2gray
from skimage import filters
from skimage import color, data, restoration

import matplotlib.pyplot as mt
import numpy as np

from skimage import feature
from skimage.filters import sobel
from skimage.morphology import watershed
import cv2

from scipy.misc import imsave
import operator
import glob
import os

# Denoising Image 

In [6]:
def denoise(imgName):
    im=io.imread(imgName)    
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    denoised = filters.median(im, selem=np.ones((10, 2)))
#     mt.imshow(denoised,cmap='gray')
#     mt.show()
    elevation_map = sobel(denoised)
    markers = np.zeros_like(denoised)
    markers[denoised <= 210] = 1
    markers[denoised > 230] = 2
    segmentation = watershed(elevation_map, markers)
    
#     mt.imshow(segmentation)
#     mt.show()
    
#     mt.imshow(segmentation,cmap='gray')
#     mt.show()
    newImg=segmentation
    newImg[newImg == 1] = 0
    newImg[newImg == 2] = 255
#     newImg = newImg.astype(np.uint8)
#     print('-------------')
#     print(np.unique(newImg))
#     mt.imshow(newImg)
#     mt.show()
#     print(imgName[10:])
#     print(denoised.shape)
    #fname='./denoised output/'+imgName[10:]
    #io.imsave(fname,denoised)
#     mt.imshow(denoised,cmap='gray')
#     return denoised
    
    return newImg

# Segmentation - Grid Based Clustering

In [8]:
def bfs(grids, i, visited, grids_set):
    queue = [(i, grids[i])]
    character = []
    while queue:
        node = queue.pop(0)
        ind, cord = node[0], node[1]
        visited[ind] = 1
        character.append(cord)
        neightbor_1 = (cord[0], cord[1] + 1)
        neightbor_2 = (cord[0] + 1, cord[1])
        neightbor_3 = (cord[0], cord[1] - 1)
        neightbor_4 = (cord[0] - 1, cord[1])
        if neightbor_1 in grids_set:
            ind1 = grids.index(neightbor_1)
            if visited[ind1] == 0 and (ind1, neightbor_1) not in queue:
                queue.append((ind1, neightbor_1))
        if neightbor_2 in grids_set:
            ind2 = grids.index(neightbor_2)
            if visited[ind2] == 0 and (ind2, neightbor_2) not in queue:
                queue.append((ind2, neightbor_2))
        if neightbor_3 in grids_set:
            ind3 = grids.index(neightbor_3)
            if visited[ind3] == 0 and (ind3, neightbor_3) not in queue:
                queue.append((ind3, neightbor_3))
        if neightbor_4 in grids_set:
            ind4 = grids.index(neightbor_4)
            if visited[ind4] == 0 and (ind4, neightbor_4) not in queue:
                queue.append((ind4, neightbor_4))
    return character

In [9]:
def get_chars_as_clusters(img,no_of_char):# returns character clusters -4 chars
    #img = io.imread(file)
    grids = []
    for col in range(img.shape[1]):  # column size
        for row in range(img.shape[0]):  # row size
            if img[row][col] < 175:
                grids.append((row, col))
    graphs = []
    visited = [0 for i in range(len(grids))]
    grids_set = set(grids)
    for i in range(len(grids)):
        if visited[i] == 0:
            graph = bfs(grids, i, visited, grids_set)
            graphs.append((len(graph), graph, i))
            
    graphs.sort(reverse=True)
    graphs = graphs[:no_of_char] 

    return graphs

In [10]:
def segment_image(denoised_img, img_file_location, no_of_char):
    characters = get_chars_as_clusters(denoised_img,no_of_char)
    characters.sort(key=operator.itemgetter(2))
    i = 0
    for char in characters:
        result_img = np.zeros([128, 128, 3], dtype=np.uint8)
        result_img.fill(255)
        char[1].sort(key=operator.itemgetter(1))
        width_margin = 19 - char[1][0][1]
        for cord in char[1]:
            if (cord[1] + width_margin) < 128:
                result_img[cord[0], cord[1] + width_margin] = 0
        folder_path = "./Test" + str(no_of_char) + "char/" + img_file_location.split('/')[-1].split('.')[0] 
        if not os.path.exists(folder_path):
            #print(folder_path + " already exists")
            os.makedirs(folder_path)
        result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2GRAY)
        imsave(folder_path + "/Char"  + str(i) + ".jpg", result_img)
        i += 1

In [11]:
def generate_denoised_segmented_image(img_file_location,no_of_char):
    img = denoise(img_file_location)
    segment_image(img, img_file_location,no_of_char) # Resulting segments are put in segments folder

In [36]:
# for 4 characters training data
def generate_for_4char_dataset():
    for path in glob.glob("./Dataset4char/*.png"):
        generate_denoised_segmented_image(path,4)
        
generate_for_4char_dataset()

./Test4char/ZXTX already exists


/home/dhruv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


./Test4char/FBDU already exists
./Test4char/BOPT already exists
./Test4char/FYXG already exists
./Test4char/IEBN already exists
./Test4char/TPDK already exists
./Test4char/QYNF already exists
./Test4char/ETTT already exists
./Test4char/KPCV already exists
./Test4char/DEEO already exists
./Test4char/PYAS already exists
./Test4char/IEJX already exists
./Test4char/JLCO already exists
./Test4char/IWNS already exists
./Test4char/KWOW already exists
./Test4char/OYOG already exists
./Test4char/IQNQ already exists
./Test4char/CNMT already exists
./Test4char/UEKT already exists
./Test4char/OYNP already exists
./Test4char/BEYM already exists
./Test4char/XZJO already exists
./Test4char/ATUM already exists
./Test4char/RVGS already exists
./Test4char/IFZM already exists
./Test4char/XAKL already exists
./Test4char/OYIO already exists
./Test4char/OMEA already exists
./Dataset4char/QEEJ.png
./Dataset4char/QEEJ.png
./Dataset4char/QEEJ.png
./Dataset4char/QEEJ.png
./Dataset4char/QEEJ.png
./Dataset4char/Q

./Test4char/KWIJ already exists
./Test4char/HGJK already exists
./Test4char/DYNQ already exists
./Test4char/CMBX already exists
./Test4char/SEPE already exists
./Test4char/KMAY already exists
./Test4char/FPNQ already exists
./Test4char/THNV already exists
./Test4char/VSQC already exists
./Test4char/QABP already exists
./Test4char/DAAV already exists
./Test4char/SIWM already exists
./Test4char/CRBF already exists
./Test4char/YUWO already exists
./Test4char/YGIC already exists
./Test4char/PUGS already exists
./Test4char/PHJS already exists
./Test4char/TGID already exists
./Test4char/KVBF already exists
./Test4char/SQUA already exists
./Test4char/DMAF already exists
./Test4char/BSGF already exists
./Test4char/MFLN already exists
./Test4char/WWPI already exists
./Test4char/MHNK already exists
./Test4char/NQKS already exists
./Test4char/ILFE already exists
./Test4char/HIES already exists
./Test4char/HELF already exists
./Test4char/FVMQ already exists
./Test4char/OIUE already exists
./Test4c

./Test4char/HIGL already exists
./Test4char/LWEV already exists
./Test4char/IXAY already exists
./Test4char/UYQU already exists
./Test4char/XMIJ already exists
./Test4char/EUXP already exists
./Test4char/MSRL already exists
./Test4char/HLGU already exists
./Test4char/MGHD already exists
./Test4char/EUMA already exists
./Test4char/WYNT already exists
./Test4char/JVKW already exists
./Test4char/YKYP already exists
./Test4char/NNNG already exists
./Test4char/USJM already exists
./Test4char/HFXG already exists
./Test4char/RLGI already exists
./Test4char/XMRG already exists
./Test4char/ITZM already exists
./Test4char/VXFB already exists
./Test4char/SBGF already exists
./Test4char/OXVW already exists
./Test4char/OKQH already exists
./Test4char/NYKZ already exists
./Test4char/FSSW already exists
./Test4char/FVWF already exists
./Test4char/FCBG already exists
./Test4char/AGJX already exists
./Test4char/ACOE already exists
./Test4char/EJSS already exists
./Test4char/ELOF already exists
./Test4c

./Test4char/MLOS already exists
./Test4char/UJBE already exists
./Test4char/KOAD already exists
./Test4char/NTHC already exists
./Test4char/CCNN already exists
./Test4char/NCIH already exists
./Test4char/ASQM already exists
./Test4char/QEYL already exists
./Test4char/RFBC already exists
./Test4char/ZQHD already exists
./Test4char/NMZM already exists
./Test4char/GVJM already exists
./Test4char/DWCP already exists
./Test4char/OUYP already exists
./Test4char/GEUQ already exists
./Test4char/KOLA already exists
./Test4char/GKZN already exists
./Test4char/UDCG already exists
./Test4char/PQVE already exists
./Test4char/KPIL already exists
./Test4char/SNWS already exists
./Test4char/ECTD already exists
./Test4char/HGJE already exists
./Test4char/PYBX already exists
./Test4char/ECWV already exists
./Test4char/RLNK already exists
./Test4char/DIWW already exists
./Test4char/BZJK already exists
./Test4char/CSDS already exists
./Test4char/RSXW already exists
./Test4char/LYLQ already exists
./Test4c

./Test4char/JXDX already exists
./Test4char/QVVR already exists
./Test4char/THOM already exists
./Test4char/UIXG already exists
./Test4char/FGTV already exists
./Test4char/UERF already exists
./Test4char/CMWA already exists
./Test4char/VVVN already exists
./Test4char/COAK already exists
./Test4char/DVXP already exists
./Test4char/OVHC already exists
./Test4char/XDAP already exists
./Test4char/IXUG already exists
./Test4char/XUFC already exists
./Test4char/SODF already exists
./Test4char/GRXX already exists
./Test4char/REUS already exists
./Test4char/LEWN already exists
./Test4char/ERQU already exists
./Test4char/XDVT already exists
./Test4char/YCOG already exists
./Test4char/YLDC already exists
./Test4char/AQHL already exists
./Test4char/NFEP already exists
./Test4char/KPSJ already exists
./Dataset4char/KPSJ.png
./Dataset4char/KPSJ.png
./Dataset4char/KPSJ.png
./Dataset4char/KPSJ.png
./Dataset4char/KPSJ.png
./Dataset4char/KPSJ.png
./Dataset4char/KPSJ.png
./Dataset4char/KPSJ.png
./Datase

./Test4char/KZZT already exists
./Test4char/HGAZ already exists
./Test4char/PGRF already exists
./Test4char/VPAE already exists
./Test4char/YDPA already exists
./Test4char/PVJF already exists
./Test4char/GEYO already exists
./Test4char/OBQU already exists
./Test4char/TABD already exists
./Test4char/ISHY already exists
./Test4char/ARXX already exists
./Test4char/EPQA already exists
./Test4char/OPZG already exists
./Test4char/OVHL already exists
./Test4char/ABND already exists
./Test4char/VTJR already exists
./Test4char/XGGX already exists
./Test4char/PIRP already exists
./Test4char/DNYJ already exists
./Test4char/HGLE already exists
./Test4char/OGAF already exists
./Test4char/FXKR already exists
./Test4char/ZGVA already exists
./Test4char/MPFJ already exists
./Test4char/ICIW already exists
./Test4char/XWSR already exists
./Test4char/OPSU already exists
./Test4char/TSLW already exists
./Test4char/GYWE already exists
./Test4char/SFTY already exists
./Test4char/DVEE already exists
./Test4c

./Test4char/ZBJO already exists
./Test4char/UGYF already exists
./Test4char/HYTV already exists
./Test4char/YGSW already exists
./Test4char/YTPS already exists
./Test4char/LBJN already exists
./Test4char/YMKT already exists
./Test4char/HUKZ already exists
./Test4char/GOJW already exists
./Test4char/TSGU already exists
./Test4char/HESJ already exists
./Test4char/YIAK already exists
./Test4char/XTWI already exists
./Test4char/QVQC already exists
./Test4char/CXPA already exists
./Test4char/MEPL already exists
./Test4char/NLZW already exists
./Test4char/AMWK already exists
./Test4char/KVEI already exists
./Test4char/GETF already exists
./Test4char/VRXD already exists
./Test4char/LQNF already exists
./Test4char/QMAU already exists
./Test4char/UMJN already exists
./Test4char/WKRZ already exists
./Test4char/CJFG already exists
./Test4char/FCIN already exists
./Test4char/SMWL already exists
./Test4char/DMHQ already exists
./Test4char/UTKZ already exists
./Test4char/UCRX already exists
./Test4c

In [12]:
# for 1 characters training data
def generate_for_1char_dataset():
    for path in glob.glob("./Dataset1char/*.png"):
        generate_denoised_segmented_image(path,1)
        
generate_for_1char_dataset()

/home/dhruv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [18]:
# for 1 char Training data 
def new_segment_image(denoised_img, img_file_location, no_of_char,img_no):
    characters = get_chars_as_clusters(denoised_img,no_of_char)
    characters.sort(key=operator.itemgetter(2))
    i = 0
    for char in characters:
        result_img = np.zeros([128, 128, 3], dtype=np.uint8)
        result_img.fill(255)
        char[1].sort(key=operator.itemgetter(1))
        width_margin = 19 - char[1][0][1]
        for cord in char[1]:
            if (cord[1] + width_margin) < 128:
                result_img[cord[0], cord[1] + width_margin] = 0
        folder_path = "./Train_data/" + img_file_location.split('/')[-1].split('.')[0][0]
        if not os.path.exists(folder_path):
            #print(folder_path + " already exists")
            os.makedirs(folder_path)
        result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2GRAY)
        imsave(folder_path + "/"  + str(img_no) + ".jpg", result_img)
        i += 1

In [19]:
# for 1 char Training data 
def generate_for_1char_dataset():
    img_no = 0
    for path in glob.glob("./Dataset/*.png"):
        img = denoise(path)
        new_segment_image(img, path, 1, img_no)
        img_no += 1
        
generate_for_1char_dataset()

/home/dhruv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
